In [1]:
import sys
sys.path.append('/work/pikachu/utils/')
sys.path.append('/work/pikachu/third/')
sys.path.append('..')
from gezi.common import *
import gezi
from src.config import *
from src.util import *
gezi.init_flags()

<IPython.core.display.Javascript object>

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[07/09/23 21:09:54] config.py:99 in <module>- VOCAB_SIZE: 65
[07/09/23 21:09:54] config.py:100 in <module>- len(IDX2CHAR): 65


In [2]:
!ls $FLAGS.root

character_to_prediction_index.json  supplemental_landmarks     train.csv
inference_args.json		    supplemental_metadata.csv  train_landmarks
means.npy			    tfrecords
stds.npy			    train2.csv


In [3]:
train = pd.read_csv(f'{FLAGS.root}/train.csv')
train

path     file_id  sequence_id  \
0         train_landmarks/5414471.parquet     5414471   1816796431   
1         train_landmarks/5414471.parquet     5414471   1816825349   
2         train_landmarks/5414471.parquet     5414471   1816909464   
3         train_landmarks/5414471.parquet     5414471   1816967051   
4         train_landmarks/5414471.parquet     5414471   1817123330   
...                                   ...         ...          ...   
67203  train_landmarks/2118949241.parquet  2118949241    388192924   
67204  train_landmarks/2118949241.parquet  2118949241    388225542   
67205  train_landmarks/2118949241.parquet  2118949241    388232076   
67206  train_landmarks/2118949241.parquet  2118949241    388235284   
67207  train_landmarks/2118949241.parquet  2118949241    388332538   

       participant_id                          phrase  
0                 217                    3 creekhouse  
1                 107                 scales/kuhaylah  
2                   1             1383 william lanier  
3                  63               988 franklin lane  
4                  89       6920 northeast 661st road  
...               ...                             ...  
67203              88                    431-366-2913  
67204             154                    994-392-3850  
67205              95  https://www.tianjiagenomes.com  
67206              36               90 kerwood circle  
67207             176                      802 co 66b  

[67208 rows x 5 columns]

In [4]:
train.groupby('participant_id')['file_id'].apply(lambda x: set(list(x))).reset_index()

participant_id                                            file_id
0                0  {1643479812, 1019715464, 1133664520, 156223463...
1                1  {1643479812, 1019715464, 1133664520, 156223463...
2                2  {1643479812, 1019715464, 1133664520, 156223463...
3                4  {1643479812, 1019715464, 1133664520, 156223463...
4                6  {1643479812, 1019715464, 1133664520, 156223463...
..             ...                                                ...
89             241  {1643479812, 1019715464, 1133664520, 156223463...
90             242  {1643479812, 1019715464, 1133664520, 156223463...
91             246  {1643479812, 1019715464, 1133664520, 156223463...
92             251  {1643479812, 1019715464, 1133664520, 156223463...
93             254  {1643479812, 1019715464, 1133664520, 156223463...

[94 rows x 2 columns]

In [5]:
train[train.file_id==1643479812]

path     file_id  sequence_id  \
48231  train_landmarks/1643479812.parquet  1643479812    388334418   
48232  train_landmarks/1643479812.parquet  1643479812    388367547   
48233  train_landmarks/1643479812.parquet  1643479812    388374408   
48234  train_landmarks/1643479812.parquet  1643479812    388393623   
48235  train_landmarks/1643479812.parquet  1643479812    388439266   
...                                   ...         ...          ...   
49224  train_landmarks/1643479812.parquet  1643479812    418903633   
49225  train_landmarks/1643479812.parquet  1643479812    418965744   
49226  train_landmarks/1643479812.parquet  1643479812    418970196   
49227  train_landmarks/1643479812.parquet  1643479812    418972653   
49228  train_landmarks/1643479812.parquet  1643479812    418977355   

       participant_id                         phrase  
48231             153                  /mari-grayson  
48232             230             +40-019-319-138-55  
48233              56            5774 springhill b c  
48234              95                 +45-56-485-780  
48235             138               banana-residence  
...               ...                            ...  
49224             192                 +39-3564-65270  
49225              76        asiantv4u.com/sd-gordon  
49226              25                    tanya bravo  
49227              88  www.parkrecord.com/tokehanabi  
49228              70                         26636/  

[998 rows x 5 columns]

In [6]:
len(train), len(train.file_id.unique()), len(train.sequence_id.unique()), len(train.participant_id.unique())

(67208, 68, 67208, 94)

In [7]:
train['phrase_len'] = train.phrase.apply(len)

In [8]:
train.describe(PERCENTILES2)

file_id   sequence_id  participant_id    phrase_len
count  6.720800e+04  6.720800e+04    67208.000000  67208.000000
mean   1.094448e+09  1.072696e+09      119.758154     17.804636
std    6.395616e+08  6.177372e+08       74.330468      5.727504
min    5.414471e+06  7.109500e+04        0.000000      1.000000
25%    5.277082e+08  5.376519e+08       63.000000     12.000000
50%    1.099408e+09  1.074387e+09      113.000000     17.000000
75%    1.662743e+09  1.605592e+09      178.000000     22.000000
90%    1.969986e+09  1.930091e+09      230.000000     27.000000
95%    2.036581e+09  2.038967e+09      239.000000     28.000000
99%    2.118949e+09  2.125348e+09      254.000000     30.000000
99.9%  2.118949e+09  2.145613e+09      254.000000     30.000000
max    2.118949e+09  2.147465e+09      254.000000     31.000000

In [9]:
from gezi import plot
gezi.plot.his_counts(train, 'phrase_len',32)

In [10]:
gezi.plot.pie_counts(train, 'phrase_len')

In [11]:
train[train.phrase_len == 12].head(100)

path  file_id  sequence_id  participant_id  \
0    train_landmarks/5414471.parquet  5414471   1816796431             217   
8    train_landmarks/5414471.parquet  5414471   1817195757             136   
25   train_landmarks/5414471.parquet  5414471   1817672186              74   
34   train_landmarks/5414471.parquet  5414471   1817895313             181   
39   train_landmarks/5414471.parquet  5414471   1818029273             217   
..                               ...      ...          ...             ...   
543  train_landmarks/5414471.parquet  5414471   1833049065             181   
546  train_landmarks/5414471.parquet  5414471   1833181503             147   
547  train_landmarks/5414471.parquet  5414471   1833196584               1   
552  train_landmarks/5414471.parquet  5414471   1833341657             216   
555  train_landmarks/5414471.parquet  5414471   1833426980             227   

           phrase  phrase_len  
0    3 creekhouse          12  
8    242-197-6202          12  
25   596-033-4046          12  
34   620-510-6135          12  
39   6738 co 5670          12  
..            ...         ...  
543  rebekah choi          12  
546  528-667-9231          12  
547  479-985-8668          12  
552  571-385-2313          12  
555  836-640-0626          12  

[100 rows x 6 columns]

In [12]:
train['phrase_type'] = train['phrase'].apply(get_phrase_type)

In [13]:
gezi.plot.pie_counts(train, 'phrase_type')

In [14]:
gezi.plot.his_counts(train[train.phrase_type=='address'], 'phrase_len',32)

In [15]:
gezi.plot.pie_counts(train[train.phrase_type=='address'], 'phrase_len')

In [16]:
gezi.plot.his_counts(train[train.phrase_type=='url'], 'phrase_len',32)

In [17]:
gezi.plot.pie_counts(train[train.phrase_type=='url'], 'phrase_len')

In [18]:
gezi.plot.his_counts(train[train.phrase_type=='phone_number'], 'phrase_len', 32)

In [19]:
gezi.plot.pie_counts(train[train.phrase_type=='phone_number'], 'phrase_len')

In [20]:
train['file_path'] = FLAGS.root + '/' + train.path

In [21]:
train.groupby('phrase')['participant_id'].nunique().sort_values(ascending=False)

phrase
surprise az                        16
yonkers new york                   15
fort worth tx                      15
raleigh north carolina             14
north charleston south carolina    13
                                   ..
861-913-0326                        1
861-868-0963                        1
861-764-7248                        1
861-461-4712                        1
+20-0026-4265-973                   1
Name: participant_id, Length: 46478, dtype: int64

In [22]:
train.groupby('phrase')['participant_id'].nunique().reset_index().describe()

participant_id
count    46478.000000
mean         1.442855
std          0.705419
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         16.000000

In [23]:
# Read First Parquet File
example_parquet_df = pd.read_parquet(train['file_path'][0])

# Each parquet file contains 1000 recordings
print(f'# Unique Recording: {example_parquet_df.index.nunique()}')
# Display DataFrame layout
example_parquet_df

# Unique Recording: 1000


frame  x_face_0  x_face_1  x_face_2  x_face_3  x_face_4  \
sequence_id                                                            
1816796431       0  0.710588  0.699951  0.705657  0.691768  0.699669   
1816796431       1  0.709525  0.697582  0.703713  0.691016  0.697576   
1816796431       2  0.711059  0.700858  0.706272  0.693285  0.700825   
1816796431       3  0.712799  0.702518  0.707840  0.694899  0.702445   
1816796431       4  0.712349  0.705451  0.709918  0.696006  0.705180   
...            ...       ...       ...       ...       ...       ...   
1848182207     296  0.657136  0.635888  0.643259  0.619031  0.633084   
1848182207     297  0.655706  0.635570  0.642730  0.618637  0.632830   
1848182207     298  0.653681  0.636057  0.643054  0.618643  0.633258   
1848182207     299  0.654293  0.635543  0.642558  0.617969  0.632699   
1848182207     300  0.655109  0.634491  0.641743  0.617779  0.631771   

             x_face_5  x_face_6  x_face_7  x_face_8  ...  z_right_hand_11  \
sequence_id                                          ...                    
1816796431   0.701980  0.709724  0.610405  0.712660  ...        -0.245855   
1816796431   0.700467  0.709796  0.616540  0.713729  ...              NaN   
1816796431   0.703319  0.711549  0.615606  0.715143  ...              NaN   
1816796431   0.704794  0.712483  0.625044  0.715677  ...        -0.370770   
1816796431   0.706928  0.712685  0.614356  0.714875  ...              NaN   
...               ...       ...       ...       ...  ...              ...   
1848182207   0.631827  0.630708  0.533120  0.626672  ...        -0.143147   
1848182207   0.631554  0.630344  0.531868  0.626445  ...              NaN   
1848182207   0.631800  0.630059  0.531178  0.625990  ...              NaN   
1848182207   0.631167  0.629263  0.531019  0.625069  ...              NaN   
1848182207   0.630562  0.629580  0.533534  0.626065  ...        -0.196495   

             z_right_hand_12  z_right_hand_13  z_right_hand_14  \
sequence_id                                                      
1816796431         -0.269148        -0.129743        -0.251501   
1816796431               NaN              NaN              NaN   
1816796431               NaN              NaN              NaN   
1816796431         -0.408097        -0.185217        -0.325494   
1816796431               NaN              NaN              NaN   
...                      ...              ...              ...   
1848182207         -0.139659        -0.066276        -0.130910   
1848182207               NaN              NaN              NaN   
1848182207               NaN              NaN              NaN   
1848182207               NaN              NaN              NaN   
1848182207         -0.202258        -0.101019        -0.179127   

             z_right_hand_15  z_right_hand_16  z_right_hand_17  \
sequence_id                                                      
1816796431         -0.278687        -0.266530        -0.152852   
1816796431               NaN              NaN              NaN   
1816796431               NaN              NaN              NaN   
1816796431         -0.343373        -0.328294        -0.203126   
1816796431               NaN              NaN              NaN   
...                      ...              ...              ...   
1848182207         -0.127341        -0.106674        -0.083439   
1848182207               NaN              NaN              NaN   
1848182207               NaN              NaN              NaN   
1848182207               NaN              NaN              NaN   
1848182207         -0.182757        -0.169923        -0.116275   

             z_right_hand_18  z_right_hand_19  z_right_hand_20  
sequence_id                                                     
1816796431         -0.257519        -0.275822        -0.266876  
1816796431               NaN              NaN              NaN  
1816796431               NaN              NaN              NaN  
1816796431         -0.315719   

In [24]:
list(example_parquet_df.columns)

['frame',
 'x_face_0',
 'x_face_1',
 'x_face_2',
 'x_face_3',
 'x_face_4',
 'x_face_5',
 'x_face_6',
 'x_face_7',
 'x_face_8',
 'x_face_9',
 'x_face_10',
 'x_face_11',
 'x_face_12',
 'x_face_13',
 'x_face_14',
 'x_face_15',
 'x_face_16',
 'x_face_17',
 'x_face_18',
 'x_face_19',
 'x_face_20',
 'x_face_21',
 'x_face_22',
 'x_face_23',
 'x_face_24',
 'x_face_25',
 'x_face_26',
 'x_face_27',
 'x_face_28',
 'x_face_29',
 'x_face_30',
 'x_face_31',
 'x_face_32',
 'x_face_33',
 'x_face_34',
 'x_face_35',
 'x_face_36',
 'x_face_37',
 'x_face_38',
 'x_face_39',
 'x_face_40',
 'x_face_41',
 'x_face_42',
 'x_face_43',
 'x_face_44',
 'x_face_45',
 'x_face_46',
 'x_face_47',
 'x_face_48',
 'x_face_49',
 'x_face_50',
 'x_face_51',
 'x_face_52',
 'x_face_53',
 'x_face_54',
 'x_face_55',
 'x_face_56',
 'x_face_57',
 'x_face_58',
 'x_face_59',
 'x_face_60',
 'x_face_61',
 'x_face_62',
 'x_face_63',
 'x_face_64',
 'x_face_65',
 'x_face_66',
 'x_face_67',
 'x_face_68',
 'x_face_69',
 'x_face_70',
 'x_fa

In [25]:
SEED = 42
# Number of Unique Frames in Recording
N_UNIQUE_FRAMES = []

UNIQUE_FILE_PATHS = pd.Series(train['file_path'].unique())

for idx, file_path in enumerate(tqdm(UNIQUE_FILE_PATHS)):
    df = pd.read_parquet(file_path)
    for group, group_df in df.groupby('sequence_id'):
        N_UNIQUE_FRAMES.append(group_df['frame'].nunique())

# Convert to Numpy Array
N_UNIQUE_FRAMES = np.array(N_UNIQUE_FRAMES)

  0%|          | 0/68 [00:00<?, ?it/s]

In [26]:
gezi.plot.his_counts(pd.Series(N_UNIQUE_FRAMES).reset_index(name='frames'), 'frames')

In [27]:
example_parquet_df.groupby('sequence_id')['frame'].nunique().describe(PERCENTILES2)

count    1000.000000
mean      162.699000
std        88.006031
min         1.000000
25%       104.000000
50%       148.500000
75%       208.250000
90%       278.000000
95%       322.000000
99%       417.090000
99.9%     584.088000
max       672.000000
Name: frame, dtype: float64

In [28]:
metadata = pd.read_csv(f'{FLAGS.root}/supplemental_metadata.csv')
metadata

path     file_id  sequence_id  \
0        supplemental_landmarks/33432165.parquet    33432165   1535467051   
1        supplemental_landmarks/33432165.parquet    33432165   1535499058   
2        supplemental_landmarks/33432165.parquet    33432165   1535530550   
3        supplemental_landmarks/33432165.parquet    33432165   1535545499   
4        supplemental_landmarks/33432165.parquet    33432165   1535585216   
...                                          ...         ...          ...   
52953  supplemental_landmarks/2100073719.parquet  2100073719   1090866442   
52954  supplemental_landmarks/2100073719.parquet  2100073719   1090966452   
52955  supplemental_landmarks/2100073719.parquet  2100073719   1091005846   
52956  supplemental_landmarks/2100073719.parquet  2100073719   1091011550   
52957  supplemental_landmarks/2100073719.parquet  2100073719   1091039755   

       participant_id                                    phrase  
0                 251         coming up with killer sound bites  
1                 239                we better investigate this  
2                 245          interesting observation was made  
3                  38              victims deserve more redress  
4                 254  knee bone is connected to the thigh bone  
...               ...                                       ...  
52953             239                 want to join us for lunch  
52954              95          this phenomenon will never occur  
52955              40                    the winner of the race  
52956             241                are you sure you want this  
52957             122       every saturday he folds the laundry  

[52958 rows x 5 columns]

In [29]:
np.array(list(train["path"].value_counts().to_dict().values())).min()
train["path"].describe().to_frame().T

count unique                              top  freq
path  67208     68  train_landmarks/5414471.parquet  1000

: 